In [1]:
#Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
import xml.etree.ElementTree as ET
import plotly.express as px
from skimage import io
from glob import glob
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split

### Labeling 

#### Parsing data from XML and converting it into CSV

In [2]:
path = glob('../Images/annotations/*.xml')
labels_dict = dict(filepath=[], x_min=[], y_min=[], x_max=[], y_max=[]) #Save in dict because it is easier to convert into df

In [3]:
tree = ET.parse(path[0])
root = tree.getroot()

In [4]:
#Grabs all the box annotation data and appends it into the list, which is inside 'labels_dict' dictionary
for filename in path:
    tree = ET.parse(filename)
    root = tree.getroot()
    member_object = root.find('object')
    labels_info = member_object.find('bndbox')
    xmin = int(labels_info.find('xmin').text)
    xmax = int(labels_info.find('xmax').text)
    ymin = int(labels_info.find('ymin').text)
    ymax = int(labels_info.find('ymax').text)

    labels_dict['filepath'].append(filename)
    labels_dict['x_min'].append(xmin)
    labels_dict['y_min'].append(ymin)
    labels_dict['x_max'].append(xmax)
    labels_dict['y_max'].append(ymax)

In [5]:
df = pd.DataFrame(labels_dict)
df.head()

,filepath,x_min,y_min,x_max,y_max
0,../Images/annotations/Cars213.xml,150,160,233,194
1,../Images/annotations/Cars207.xml,254,229,372,264
2,../Images/annotations/Cars398.xml,72,139,196,222
3,../Images/annotations/Cars367.xml,224,161,382,222
4,../Images/annotations/Cars401.xml,158,223,238,240


In [6]:
#To save it in the project folder 
df.to_csv('../Data/labels.csv')

With this code we have successfully extracted the box diagonal positions of each image and converted the data from an unstructured format into a structured format.

In [7]:
filename

'../Images/annotations/Cars236.xml'

In [8]:
filename = df['filepath'][0]
def get_file_name(filename):
    filename_img = ET.parse(filename).getroot().find('filename').text
    filename_loc = '../Images/images/' + filename_img
    return filename_loc
get_file_name(filename)

'../Images/images/Cars213.png'

In [9]:
#Applys the function and retrieves the corresponding image location in the project folder
image_path = list(df['filepath'].apply(get_file_name))

In [10]:
#Checks if the images are in order similar to the dataframe 'df'
image_path[:5]

['../Images/images/Cars213.png',
 '../Images/images/Cars207.png',
 '../Images/images/Cars398.png',
 '../Images/images/Cars367.png',
 '../Images/images/Cars401.png']

#### Verifying the data

In [11]:
x_min,y_min,x_max,y_max = df.iloc[20,1:].values
print(f'X_min : {x_min}')
print(f'Y_min : {y_min}')
print(f'X_max : {x_max}')
print(f'Y_max : {y_max}')

X_min : 226
Y_min : 181
X_max : 327
Y_max : 210


In [14]:
file_path = image_path[0]
img = io.imread(file_path)
fig = px.imshow(img)
#xmin=150, ymin=160, xmax=233, ymax=194
fig.update_layout(width=600, height=500, margin=dict(l=10, r=10, b=10, t=10),xaxis_title='Figure 20 - Cars52.jpeg with bounding box')
fig.add_shape(type='rect',x0=150, x1=233, y0=160, y1=194, xref='x', yref='y',line_color='red')

### Data Preprocessing 
#### Read Data

In [28]:
labels = df.iloc[:,1:].values
data = []
output = []
for ind in range(len(image_path)):
    image = image_path[ind]
    img_arr = cv2.imread(image)
    h,w,d = img_arr.shape
    # Prepprocesing
    load_image = load_img(image,target_size=(224,224))
    load_image_arr = img_to_array(load_image)
    norm_load_image_arr = load_image_arr/255.0 # Normalization
    # Normalization to labels
    xmin,ymin,xmax,ymax = labels[ind]
    nxmin,nxmax = xmin/w,xmax/w
    nymin,nymax = ymin/h,ymax/h
    label_norm = (nxmin,nymin,nxmax,nymax) # Normalized output
    # Append
    data.append(norm_load_image_arr)
    output.append(label_norm)

In [29]:
output[:5]

[(0.375, 0.5333333333333333, 0.5825, 0.6466666666666666),
 (0.635, 0.4790794979079498, 0.93, 0.5523012552301255),
 (0.18, 0.4633333333333333, 0.49, 0.74),
 (0.49777777777777776,
  0.5384615384615384,
  0.8488888888888889,
  0.7424749163879598),
 (0.395, 0.7433333333333333, 0.595, 0.8)]

In [30]:
len(data),len(output)

(433, 433)

Splitting train and test set

In [31]:
#Converting data to numpy array
X = np.array(data)
y = np.array(output)

In [32]:
#Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((346, 224, 224, 3), (346, 4), (87, 224, 224, 3), (87, 4))

In [18]:
# saving the array locally to be able to use it in other notebook(s)
np.save('../arrays/X_train',X_train)
np.save('../arrays/y_train',y_train)
np.save('../arrays/X_test',X_test)
np.save('../arrays/y_test',y_test)